# NBA Season

### Data Initialization

We are pulling the NBA season stats throughout the years from the URL of https://www.kaggle.com/datasets/justinas/nba-players-data/data

In [331]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
%matplotlib inline

**Get the kaggle dataset**

In [332]:
# read a csv file into a df
playerData = pd.read_csv('nba.csv')

teamNames = pd.read_csv('unique_teams.csv')

playerData.head()

,Unnamed: 0,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,0,Randy Livingston,HOU,22.0,193.04,94.800728,Louisiana State,USA,1996,2,...,3.9,1.5,2.4,0.3,0.042,0.071,0.169,0.487,0.248,1996-97
1,1,Gaylon Nickerson,WAS,28.0,190.50,86.182480,Northwestern Oklahoma,USA,1994,2,...,3.8,1.3,0.3,8.9,0.030,0.111,0.174,0.497,0.043,1996-97
2,2,George Lynch,VAN,26.0,203.20,103.418976,North Carolina,USA,1993,1,...,8.3,6.4,1.9,-8.2,0.106,0.185,0.175,0.512,0.125,1996-97
3,3,George McCloud,LAL,30.0,203.20,102.058200,Florida State,USA,1989,1,...,10.2,2.8,1.7,-2.7,0.027,0.111,0.206,0.527,0.125,1996-97
4,4,George Zidek,DEN,23.0,213.36,119.748288,UCLA,USA,1995,1,...,2.8,1.7,0.3,-14.1,0.102,0.169,0.195,0.500,0.064,1996-97


**Put abbreviations to Cities**

Some cities have multiple as franchises have changed

Cities had to be added as ESPN win rates used cities

In [333]:
# merge the two dataframes on team_abbreviation with df and dfTeams on abbreviations
playerData = pd.merge(playerData, teamNames, left_on='team_abbreviation', right_on='abbreviations')

# drop the team_abbreviation column and abbreviations column
playerData = playerData.drop(columns=['team_abbreviation', 'abbreviations'])

In [334]:
playerData.head()

,Unnamed: 0,player_name,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,...,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,team
0,0,Randy Livingston,22.0,193.04,94.800728,Louisiana State,USA,1996,2,42,...,1.5,2.4,0.3,0.042,0.071,0.169,0.487,0.248,1996-97,Houston
1,18,Hakeem Olajuwon,34.0,213.36,115.665960,Houston,Nigeria,1984,1,1,...,9.2,3.0,6.5,0.075,0.206,0.308,0.558,0.158,1996-97,Houston
2,29,Emanual Davis,28.0,195.58,87.996848,Delaware State,USA,Undrafted,Undrafted,Undrafted,...,1.7,2.0,6.6,0.011,0.098,0.144,0.565,0.191,1996-97,Houston
3,61,Joe Stephens,24.0,200.66,95.254320,Arkansas-Little Rock,USA,Undrafted,Undrafted,Undrafted,...,1.5,0.0,-17.4,0.250,0.111,0.279,0.300,0.000,1996-97,Houston
4,97,Eddie Johnson,38.0,200.66,97.522280,Illinois,USA,1981,2,29,...,2.7,1.0,4.1,0.034,0.126,0.220,0.541,0.102,1996-97,Houston


In [335]:
# drop the unnamed column
playerData.drop('Unnamed: 0', axis=1, inplace=True)

playerData.dtypes

player_name       object
age              float64
player_height    float64
player_weight    float64
college           object
country           object
draft_year        object
draft_round       object
draft_number      object
gp                 int64
pts              float64
reb              float64
ast              float64
net_rating       float64
oreb_pct         float64
dreb_pct         float64
usg_pct          float64
ts_pct           float64
ast_pct          float64
season            object
team              object
dtype: object

In [336]:
# look for null values
playerData.isnull().sum()

player_name         0
age                 0
player_height       0
player_weight       0
college          1852
country             0
draft_year          0
draft_round         0
draft_number        0
gp                  0
pts                 0
reb                 0
ast                 0
net_rating          0
oreb_pct            0
dreb_pct            0
usg_pct             0
ts_pct              0
ast_pct             0
season              0
team                0
dtype: int64

### Data Preprocessing

We need to deal with the columns we want to keep and also all the categorial data cols of:

player_name           object

team_abbreviation     object

college               object

country               object

draft_year            object

draft_round           object

draft_number          object

season                object

#### **Drop Some of Them**

In [337]:
playerData.drop(['player_name', 'college', 'draft_year', 'draft_round', 'draft_number', 'country'], axis=1, inplace=True)

In [338]:
playerData.head()

,age,player_height,player_weight,gp,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,team
0,22.0,193.04,94.800728,64,3.9,1.5,2.4,0.3,0.042,0.071,0.169,0.487,0.248,1996-97,Houston
1,34.0,213.36,115.665960,78,23.2,9.2,3.0,6.5,0.075,0.206,0.308,0.558,0.158,1996-97,Houston
2,28.0,195.58,87.996848,13,5.0,1.7,2.0,6.6,0.011,0.098,0.144,0.565,0.191,1996-97,Houston
3,24.0,200.66,95.254320,2,1.5,1.5,0.0,-17.4,0.250,0.111,0.279,0.300,0.000,1996-97,Houston
4,38.0,200.66,97.522280,52,8.2,2.7,1.0,4.1,0.034,0.126,0.220,0.541,0.102,1996-97,Houston


In [339]:
# categorical code season col but i want to keep the original
playerData['season'] = pd.Categorical(playerData['season']).codes + 1997

# drop the rows where seasonEncoded is less than 7. This keeps the season of 03-04 and later
playerData = playerData[playerData['season'] >= 2004]

playerData.reset_index(drop=True, inplace=True)

#### **Add in Win Rates**

In [340]:
def winRateFromYear(year):

    winRateDf = pd.read_html(f'https://www.teamrankings.com/nba/stat/win-pct-all-games?date={year}-06-16')[0]

    winRateDf['Win PCT']= winRateDf[f'{year - 1}'] 

    winRateDf['season'] = year

    winRateDf = winRateDf[['Team', 'Win PCT', 'season']]

    return winRateDf

**Merge DF2 with df on Team Names**

In [341]:
def getWinRates():
    # merge the two dataframes on team with df and teams on team where season is 2004
    winRateDf = pd.DataFrame()

    for year in range(2004, 2024):
        winRateDf = pd.concat([winRateDf, winRateFromYear(year)], ignore_index=True)

    winRateDf.tail()

    return winRateDf

winRateDf = pd.read_csv('winRate.csv')


In [342]:
winRateDf.head()

,Team,Win PCT,season
0,Indiana,0.725,2004
1,San Antonio,0.685,2004
2,Minnesota,0.680,2004
3,Detroit,0.667,2004
4,LA Lakers,0.664,2004


**TODO: MERGE THE DFS**

In [343]:
updatedPlayerData = pd.merge(playerData, winRateDf, left_on=['team', 'season'], right_on=['Team', 'season'])

updatedPlayerData.drop(['Team'], axis=1, inplace=True)

updatedPlayerData.head()

,age,player_height,player_weight,gp,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,team,Win PCT
0,40.0,205.74,111.130040,7,1.3,0.7,0.3,-7.9,0.000,0.217,0.165,0.521,0.100,2004,Houston,0.529
1,33.0,200.66,122.469840,52,5.0,3.9,0.6,-5.0,0.104,0.175,0.152,0.538,0.063,2004,Houston,0.529
2,28.0,193.04,97.522280,80,15.8,4.5,3.2,1.8,0.015,0.112,0.200,0.535,0.144,2004,Houston,0.529
3,23.0,205.74,100.243832,45,3.1,1.6,0.7,2.0,0.021,0.136,0.159,0.477,0.103,2004,Houston,0.529
4,29.0,195.58,104.326160,19,0.6,1.0,0.5,-8.4,0.010,0.167,0.093,0.278,0.132,2004,Houston,0.529


In [344]:
updatedPlayerData['AVG Minutes Played'] = updatedPlayerData['gp'] * updatedPlayerData['usg_pct']

In [345]:
# Filter the df to only include the max 5 of games played from each seasonEncoded and Team
updatedPlayerData = updatedPlayerData.groupby(['team', 'season']).apply(lambda x: x.nlargest(5, 'AVG Minutes Played')).reset_index(drop=True)

updatedPlayerData.head()

,age,player_height,player_weight,gp,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,team,Win PCT,AVG Minutes Played
0,26.0,203.20,98.883056,80,18.1,4.6,3.1,-3.8,0.038,0.102,0.249,0.521,0.155,2004,Atlanta,0.342,19.920
1,26.0,187.96,81.646560,81,16.8,4.1,5.4,-4.0,0.019,0.106,0.231,0.519,0.261,2004,Atlanta,0.342,18.711
2,31.0,195.58,90.718400,80,7.5,4.1,2.9,0.8,0.071,0.158,0.191,0.510,0.240,2004,Atlanta,0.342,15.280
3,29.0,205.74,106.594120,56,10.2,3.1,0.8,-5.2,0.054,0.117,0.214,0.544,0.068,2004,Atlanta,0.342,11.984
4,29.0,185.42,86.182480,71,3.8,1.6,2.7,-9.8,0.011,0.094,0.143,0.440,0.256,2004,Atlanta,0.342,10.153


#### **Convert Player Stats into Starters' Team Stats**

In [346]:
teamData = updatedPlayerData.groupby(['season', 'team']).agg(
    ptsTotal=('pts', 'sum'),
    rebTotal=('reb', 'sum'),
    astTotal=('ast', 'sum'),
    MinutesPlayed=('AVG Minutes Played', 'sum'),
    averageAge=('age', 'mean'),
    averageHeight=('player_height', 'mean'),
    averageWeight=('player_weight', 'mean'),
    winRate=('Win PCT', 'first')
).reset_index()

teamData

,season,team,ptsTotal,rebTotal,astTotal,MinutesPlayed,averageAge,averageHeight,averageWeight,winRate
0,2004,Atlanta,56.4,17.5,14.9,76.048,28.2,195.580,92.804923,0.342
1,2004,Boston,58.8,21.7,13.2,87.015,24.0,200.152,99.608803,0.419
2,2004,Brooklyn,71.6,30.0,20.0,80.431,28.8,199.136,100.062395,0.581
3,2004,Chicago,62.5,24.9,16.1,77.090,27.8,199.644,100.788142,0.281
4,2004,Cleveland,67.7,31.5,11.6,82.232,26.0,207.772,110.313574,0.427
...,...,...,...,...,...,...,...,...,...,...
554,2023,Sacramento,87.8,26.9,21.8,85.421,26.4,200.152,95.072883,0.573
555,2023,San Antonio,64.1,19.4,17.0,67.671,23.8,194.056,93.439952,0.268
556,2023,Toronto,85.6,26.6,19.8,77.189,26.6,198.120,96.252222,0.494
557,2023,Utah,83.9,24.2,16.7,73.005,27.0,200.152,99.608803,0.451


In [347]:
winRate = teamData['winRate']

In [348]:
teamData.drop(['season','team','winRate'], axis=1, inplace=True)

teamData.head()

,ptsTotal,rebTotal,astTotal,MinutesPlayed,averageAge,averageHeight,averageWeight
0,56.4,17.5,14.9,76.048,28.2,195.580,92.804923
1,58.8,21.7,13.2,87.015,24.0,200.152,99.608803
2,71.6,30.0,20.0,80.431,28.8,199.136,100.062395
3,62.5,24.9,16.1,77.090,27.8,199.644,100.788142
4,67.7,31.5,11.6,82.232,26.0,207.772,110.313574


In [349]:
winRate.head()

0    0.342
1    0.419
2    0.581
3    0.281
4    0.427
Name: winRate, dtype: float64

In [350]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(teamData, winRate, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test_Scaled = scaler.transform(X_test)

In [351]:
model = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),  # Input layer
    keras.layers.Dense(64, activation='relu'),       # Hidden layer
    keras.layers.Dense(32, activation='relu'),       # Hidden layer
    keras.layers.Dense(1)                             # Output layer (for regression)
])

In [352]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [353]:
model.fit(X_train, y_train, epochs=100, batch_size=5, validation_split=0.2)

Epoch 1/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2017 - mae: 0.3409 - val_loss: 0.0357 - val_mae: 0.1534
Epoch 2/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0291 - mae: 0.1387 - val_loss: 0.0216 - val_mae: 0.1126
Epoch 3/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0235 - mae: 0.1251 - val_loss: 0.0194 - val_mae: 0.1084
Epoch 4/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0235 - mae: 0.1214 - val_loss: 0.0180 - val_mae: 0.1043
Epoch 5/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0161 - mae: 0.0994 - val_loss: 0.0195 - val_mae: 0.1101
Epoch 6/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0131 - mae: 0.0898 - val_loss: 0.0185 - val_mae: 0.1093
Epoch 7/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0108 - mae: 0.0810 - val_loss: 0.0199 - val_mae: 0.1111
Epoch 8/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0130 - mae: 0.0913 - val_loss: 0.0191 - val_mae: 0.1106
Epoch 9/100
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.010

In [354]:
test_loss, test_mae = model.evaluate(X_test_Scaled, y_test)
print(f'Test MAE: {test_mae:.3f}, Test Loss: {test_loss:.3f}')

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0197 - mae: 0.1102 
Test MAE: 0.115, Test Loss: 0.021


In [355]:
X_test.head(1), y_test.head(1)

(     ptsTotal  rebTotal  astTotal  MinutesPlayed  averageAge  averageHeight  \
 158      72.1      29.5      16.7         83.282        29.6        201.168   
 
      averageWeight  
 158      98.429464  ,
 158    0.679
 Name: winRate, dtype: float64)

In [361]:
# Example input for a new set of NBA players

predicted_win_rates = model.predict(X_test_Scaled)
actual_win_rates = y_test.values

# Display predictions alongside actual values
for pred, actual in zip(predicted_win_rates.flatten(), actual_win_rates):
    print(f'Predicted: {pred:.3f}, Actual: {actual:.3f}')

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Predicted: 0.590, Actual: 0.679
Predicted: 0.251, Actual: 0.339
Predicted: 0.333, Actual: 0.317
Predicted: 0.339, Actual: 0.750
Predicted: 0.393, Actual: 0.471
Predicted: 0.645, Actual: 0.511
Predicted: 0.541, Actual: 0.354
Predicted: 0.457, Actual: 0.500
Predicted: 0.545, Actual: 0.640
Predicted: 0.626, Actual: 0.792
Predicted: 0.548, Actual: 0.293
Predicted: 0.527, Actual: 0.628
Predicted: 0.519, Actual: 0.317
Predicted: 0.708, Actual: 0.725
Predicted: 0.607, Actual: 0.638
Predicted: 0.608, Actual: 0.632
Predicted: 0.733, Actual: 0.644
Predicted: 0.575, Actual: 0.725
Predicted: 0.563, Actual: 0.523
Predicted: 0.551, Actual: 0.596
Predicted: 0.599, Actual: 0.470
Predicted: 0.485, Actual: 0.563
Predicted: 0.384, Actual: 0.506
Predicted: 0.479, Actual: 0.512
Predicted: 0.327, Actual: 0.534
Predicted: 0.489, Actual: 0.613
Predicted: 0.604, Actual: 0.656
Predicted: 0.286, Actual: 0.232
Predicted: 0.330, Actual: 0.402
Predicted: 0.591, Actual: 0.582
Pr